In [1]:
import numpy as np
import pandas as pd
import rebound
import reboundx
import matplotlib.pyplot as plt

# User defines (can be set as default arguments or passed in)
user_core_density = 7874.0  # kg/m^3
user_mantle_density = 3000.0  # kg/m^3

In [2]:
sim = rebound.Simulation()
rebx = reboundx.Extras(sim)
# Load "fragmenting_collisions" as the collision resolve module
collision_resolve = rebx.load_collision_resolve("fragmenting_collisions")
rebx.add_collision_resolve(collision_resolve)

#sim.units = ('yr', 'AU', 'Msun')
sim.dt = 6.0/365.0
sim.rand_seed = 1
np.random.seed(1)
sim.integrator = "mercurius"
sim.collision = "direct"
sim.G = 39.476926421373

sim.add(m = 1)

n_pl = 30 # Number of planetesimals

# planetesimal mass range
lunar_mass  = 3.8e-8    # solar masses
earth_mass  = 3e-6      # solar masses
mass_min    = 0.5 * lunar_mass
mass_max    = 0.1 * earth_mass

# Set up minimum fragment mass
collision_resolve.params["fc_min_frag_mass"] = mass_min
collision_resolve.params["fc_particle_list_file"] = "family_tree.csv"

rho = 5.05e6   # units of solMass/AU^3, equal to ~3 g/cm^3

# Add 30 planetary embryos
for i in range(n_pl):
    # Orbital parameters
    a = np.random.uniform(0.1, 0.5) # semi-major axis in AU
    e = np.random.uniform(0.0, 0.01) # eccentricity
    inc = np.random.uniform(0.0, np.pi/180) # inclination in radians
    omega = np.random.uniform(0.0, 2.*np.pi) # argument of periapsis
    Omega = np.random.uniform(0.0, 2.*np.pi) # longitude of ascending node
    f = np.random.uniform(0.0, 2.*np.pi) # mean anomaly
    m = np.random.uniform(mass_min, mass_max) # particle mass in solar masses
    r = ((3*m)/(4 * np.pi * rho)) ** (1/3) * 10 # particle radius in AU, times 10 to ease collisions
    sim.add(m=m, a=a, e=e, inc=inc, omega=omega, Omega=Omega, f=f, r=r)

sim.move_to_com()

In [3]:
integration_time = 1e4

sim.integrate(integration_time)

Non grazing, M_rem to small. Merging collision detected. (Case C)
Non grazing, M_rem to small. Merging collision detected. (Case C)
Elastic bounce, (Case H).
Non grazing, M_rem to small. Merging collision detected. (Case C)
Merging collision detected. (Case B)
Elastic bounce, (Case I).
Non grazing, M_rem to small. Merging collision detected. (Case C)
Non grazing, M_rem to small. Merging collision detected. (Case C)
Elastic bounce, (Case I).
Elastic bounce, (Case I).
Non grazing, M_rem to small. Merging collision detected. (Case C)
Merging collision detected. (Case B)
Merging collision detected. (Case B)
Merging collision detected. (Case B)
Elastic bounce, (Case I).
Merging collision detected. (Case A)
Elastic bounce, (Case H).
Elastic bounce, (Case I).


In [8]:
col_names = ["t", "type", "new_id", "parent1_id", "parent2_id", "new_mass", "parent1_mass", "parent2_mass", "new_radius",
             "parent1_radius", "parent2_radius", "v_imp", "theta_imp"]

fam_tree = pd.read_csv("family_tree.csv", names=col_names)
fam_tree['cmf'] = np.nan

fam_tree.head()

,t,type,new_id,parent1_id,parent2_id,new_mass,parent1_mass,parent2_mass,new_radius,parent1_radius,parent2_radius,v_imp,theta_imp,cmf
0,13.03004,3,31,27,26,2.444913e-07,1.664324e-07,7.805890e-08,0.000226,0.000199,0.000155,0.260013,0.531226,NaN
1,13.84789,3,33,20,29,5.052266e-07,2.289412e-07,2.762854e-07,0.000288,0.000221,0.000236,0.442719,0.840484,NaN
2,349.66760,8,35,11,4,2.796299e-07,2.796299e-07,2.997439e-08,0.000236,0.000236,0.000112,0.293612,1.143138,NaN
3,349.66760,8,36,11,4,2.997439e-08,2.796299e-07,2.997439e-08,0.000112,0.000236,0.000112,0.293612,1.143138,NaN
4,1512.76700,3,38,25,16,2.313516e-07,1.738164e-07,5.753515e-08,0.000222,0.000202,0.000140,0.321107,0.244763,NaN


In [9]:
merge_types = [1, 2, 3]
erosion_types = [4, 5, 7, 10]
elastic_bounce_types = [6, 8, 9]

# Tracking Core Mass Fraction (CMF) of Bodies

To track the particles CMFs, we can divide the collision types into three categories: merging, bouncing, and erosion/accretion. 

## Merging

When two particles merge, we assume that their cores mix and their CMF will be the average of the parent CMFs.

$CMF_{new} = (CMF_{parent1}M_{parent1} + CMF_{parent2}M_{parent2})/(M_{parent1} + M_{parent2})$

Where M_{parent1}, M_{parent2}, CMF_{parent1}, CMF_{parent2} are the masses of parent 1 and 2, and CMF of parent 1 and 2 respectively. 

## Bouncing

When two particles go through an elastic (or inelastic) bounce, their masses and CMFs remain unchanged.

## Erosion/Accretion

There are multiple options for how to compute the CMFs of objects after an erosion or accretion event. Here, we demonstrate two examples from Marcus et al. (2010). 

### Model 1

In the first model we assume that the cores of the two bodies merge, and the escaping mass will be made from the lightest possible material. 

We will have to find the largest remnant (denoted with lr) and we will compute a CMF for the largest remnant, and a CMF for fragments. We assume that all fragments will have the same CMF. 

The largest remnant core mass ($M_{core, lr}$) will be:

$M_{core, lr} = min(M_{lr}, M_{core, avail})$

Where $ M_{core, avail} = CMF_{parent1} M_{parent1} + CMF_{parent2}  M_{parent2} $. Then, CMFs are: 

$CMF_{lr} = M_{core, lr}/M_{lr}$

$CMF_{frag} = (M_{core, avail} - M_{core, lr})/(M_{tot} - M_{lr})$

Where $M_{tot}$ is the total mass ($M_{tot} = M_{parent1} + M_{parent2}$)

### Model 2

In the second model, we first need to identify target and projectile, and see if the collision has resulted in an accretion ($Mlr > M_{target}$) or erosion ($Mlr > M_{target}$). 

In the accretion regime, core material from the projectile will be accreted into target first, followed by mantle material. Therefore, the mass of the largest remnant's core will be:

$M_{core, lr} = M_{core, target} + min(M_{core, avail} - M_{core, target}, M_{lr} - M_{target})$

In the erosion regime, mantle material is ejected from the target untill there is no more left, and then core material will be ejected:

$M_{core, lr} = min(M_{core, target}, M_{lr})$

And in all cases, CMFs will be derived as before:

$CMF_{lr} = M_{core, lr} / M_{lr}$

$CMF_{frags} = (M_{core, avail} - M_{core, lr})/(M_{tot} - M_{lr})$

In [10]:
# Choose erosion/accretion model here
# 1 for Model 1, and 2 for Model 2
erosion_accretion_model = 1

cmf_i = 0.3
tolerance = 1e-8

processed = set()  # (parent1_id, parent2_id) already handled


def get_parent_cmf(fam_tree, parent_id, cmf_i):
    parent = fam_tree.loc[fam_tree['new_id'] == parent_id]
    if parent.empty or pd.isna(parent['cmf'].iloc[0]):
        return cmf_i
    return parent['cmf'].iloc[0]


for idx, row in fam_tree.iterrows():

    key = (row['parent1_id'], row['parent2_id'])

    # Skip if this collision already processed
    if key in processed:
        continue
    processed.add(key)

    # All children of this collision (siblings)
    group = fam_tree[
        (fam_tree['parent1_id'] == key[0]) &
        (fam_tree['parent2_id'] == key[1])
    ]

    parent1_cmf = get_parent_cmf(fam_tree, key[0], cmf_i)
    parent2_cmf = get_parent_cmf(fam_tree, key[1], cmf_i)

    collision_type = group['type'].iloc[0]

    # =======================
    # ELASTIC BOUNCE
    # =======================
    if collision_type in elastic_bounce_types:
        for i, r in group.iterrows():
            if abs(r['new_mass'] - r['parent1_mass']) < tolerance:
                fam_tree.loc[i, 'cmf'] = parent1_cmf
            else:
                fam_tree.loc[i, 'cmf'] = parent2_cmf

    # =======================
    # MERGE
    # =======================
    elif collision_type in merge_types:
        for i, r in group.iterrows():
            fam_tree.loc[i, 'cmf'] = (r['parent1_mass'] * parent1_cmf + r['parent2_mass'] * parent2_cmf) / (r['parent1_mass'] + r['parent2_mass'])

    # =======================
    # EROSION AND ACCRETION
    # =======================
    elif collision_type in erosion_types:
        idx_max = group['new_mass'].idxmax()
        Mlr = group.loc[idx_max, 'new_mass']
        M_tot = group['parent1_mass'].iloc[0] + group['parent2_mass'].iloc[0]
        M_core_tot = (
            parent1_cmf * group['parent1_mass'].iloc[0] +
            parent2_cmf * group['parent2_mass'].iloc[0]
        )
        if erosion_accretion_model == 1:
            M_core_lr = min(Mlr, M_core_tot)

        if erosion_accretion_model == 2:
            if (group['parent1_mass'].iloc[0] >= group['parent2_mass'].iloc[0]):
                M_target = group['parent1_mass'].iloc[0]
                M_core_target = M_target * parent1_cmf
            else:
                M_target = group['parent2_mass'].iloc[0]
                M_core_target = M_target * parent2_cmf

            sum_M_frags = group['new_mass'].sum() - Mlr

            # Accretion
            if Mlr >= M_target:
                M_core_lr = M_core_target + min(M_core_tot - M_core_target, Mlr - M_target)
            # Erosion
            else:
                M_core_lr = min(M_core_target, Mlr)
 
        CMF_lr = abs(M_core_lr/Mlr)
        CMF_frag = abs((M_core_tot - M_core_lr)/(M_tot - Mlr))

        fam_tree.loc[idx_max, 'cmf'] = CMF_lr
        for i in group.index:
            if i != idx_max:
                fam_tree.loc[i, 'cmf'] = CMF_frag


We can look at erosions as an example:

In [11]:
erosions = fam_tree[fam_tree['type'].isin(erosion_types)]
erosions.head()

,t,type,new_id,parent1_id,parent2_id,new_mass,parent1_mass,parent2_mass,new_radius,parent1_radius,parent2_radius,v_imp,theta_imp,cmf
25,14600.51,4,74,63,71,3.883725e-07,2.313516e-07,1.766495e-07,0.000264,0.000222,0.000203,0.686252,0.407228,0.315162
26,14600.51,4,75,63,71,1.962863e-08,2.313516e-07,1.766495e-07,0.000098,0.000222,0.000203,0.686252,0.407228,0.000000
